# Este código trabaja con la API de Groq

* Creación de una Key para la conección con la API: https://console.groq.com/keys
* Documentación: https://console.groq.com/docs/quickstart

In [1]:
import pandas as pd
from io import StringIO
from groq import Groq

In [2]:
def datos():

    # Lista de comentarios
    comentarios = [
        "El producto llegó antes de lo esperado, muy satisfecho",
        "El servicio no fue lo que esperaba, bastante decepcionado",
        "Las promociones son atractivas, pero el proceso de compra es complicado",
        "La atención al cliente fue excelente, muy amables",
        "El producto es funcional, aunque no destaca en nada",
        "No recibí una respuesta clara a mi problema",
        "La calidad es buena, pero el precio es elevado",
        "Recibí exactamente lo que pedí, sin sorpresas",
        "Los precios son razonables para la calidad ofrecida",
        "El soporte fue útil, aunque tomó tiempo",
        "El producto cumple con las expectativas mínimas",
        "La experiencia fue neutral, no hay mucho que destacar",
        "Llevo varios días esperando una actualización",
        "El servicio es decente, pero hay margen de mejora",
        "La calidad del servicio es superior, muy contento",
        "El servicio fue estándar, cumplió con lo básico",
        "El tiempo de entrega fue aceptable",
        "El producto es útil, aunque esperaba algo más avanzado",
        "Muy satisfecho con el servicio, definitivamente volveré",
        "La atención al cliente fue pasable, nada especial",
        "El embalaje llegó dañado, pero el producto estaba bien",
        "El proceso de devolución es complicado",
        "La oferta era buena, pero la calidad no tanto",
        "Me respondieron rápido y resolvieron mi problema",
        "El producto no funciona como se describe",
        "El soporte técnico fue lento pero resolvieron mi problema",
        "El envío fue rápido y sin problemas",
        "La experiencia de compra fue simple y sin complicaciones",
        "El producto tiene una calidad superior, muy recomendable",
        "No recomendaría este servicio a otros",
        "El servicio cumplió con las expectativas, sin destacar",
        "El precio es alto, pero justificado por la calidad",
        "No quedé satisfecho con la atención recibida",
        "El proceso de pago fue sencillo y seguro",
        "El producto es exactamente como se describe, sin sorpresas",
        "La atención fue buena, aunque podría mejorar",
        "El producto llegó con retraso",
        "Muy contento con la rapidez y eficacia del servicio",
        "El producto tiene defectos de fabricación",
        "La calidad del producto es impresionante, superó mis expectativas",
        "La comunicación con el vendedor fue deficiente",
        "El servicio es funcional, aunque no destaca en nada",
        "No recibí el producto correcto",
        "La devolución fue sencilla y sin problemas",
        "El producto es de baja calidad, no vale el precio",
        "El proceso de compra fue confuso",
        "El equipo de soporte fue muy amable y eficiente",
        "La calidad del servicio es excepcional, muy recomendado",
        "El producto llegó en perfecto estado, muy satisfecho",
        "La atención al cliente fue inaceptable, muy poco profesional"
    ]
    
    # Creación de la estructura de datos
    data = {
        'ID': list(range(1, len(comentarios) + 1)),
        'COMENTARIO': comentarios
    }
    # Creación del DataFrame
    df = pd.DataFrame(data)
    return df

In [4]:
# Pega tu clave de API aquí
GROQ_API_KEY = '{su_llave}'

# Inicializa el cliente de Groq
client = Groq(api_key=GROQ_API_KEY)

# Supongamos que df ya está definido
df = datos()

# Definir una función para procesar cada lote
def procesar_lote(comentarios):
    # Crear el mensaje con los comentarios actuales
    mensajes = '\n'.join([f"{row['ID']}|{row['COMENTARIO']}" for _, row in comentarios.iterrows()])
    prompt = f'''
        Analiza el sentimiento, tópico y criterio de la siguiente tabla.
        El campo a revisar es "COMENTARIO". La tabla está separada por "|".

        ID|COMENTARIO
        {mensajes}

        Los sentimientos posibles son: Negativo, Neutral y Positivo.
        Los tópicos posibles son se deben poner depediendo si el contenido habla de competencia, dinero :
        - Facturación
        - Fallos en el servicio
        - Precio/Tarifas
        - Competencia
        - Calidad de servicio al cliente
        - Instalación/Entrega
        - Otros

        Los criterios deben estar relacionados con el comentario y el tópico.

        Entrega el resultado en una cadena de texto separada por "|", exclusivamente con el siguiente formato:
        ID|COMENTARIO|SENTIMIENTO|TÓPICO|CRITERIO

        Por cada comentario, deja un salto de línea.

        Solo proporciona la respuesta en el formato indicado.
    '''

    # Llamar al modelo
    response = client.chat.completions.create(
        #modelo seleccionado
        model="gemma2-9b-it",
        #temperatura del modelo (entre mas cercana a cero es mas determinista "menos imaginativo o mas apegado al prompt")
        temperature=0.5,
        # Controla la diversidad mediante "nucleus sampling": 0.5 significa que se consideran la mitad de todas las opciones, 
        #ponderadas por su probabilidad. 
        top_p=1,
        
        messages=[{
            "role": "user",
            "content": prompt
        }]
    )
    
    # Procesar la respuesta del modelo
    content = response.choices[0].message.content
    data_io = StringIO(content)
    df_lote = pd.read_csv(data_io, delimiter='|', names=['ID', 'COMENTARIO', 'SENTIMIENTO', 'TOPICO', 'CRITERIO'])
    
    return df_lote

# Dividir el DataFrame en lotes de 10
lotes = [df[i:i + 10] for i in range(0, df.shape[0], 10)]
    
# Procesar cada lote y combinar los resultados
resultados = pd.concat([procesar_lote(lote) for lote in lotes], ignore_index=True)

while resultados['CRITERIO'].isna().sum() > 0:
    # Dividir el DataFrame en lotes de 10
    lotes = [df[i:i + 10] for i in range(0, df.shape[0], 10)]
    
    # Procesar cada lote y combinar los resultados
    resultados = pd.concat([procesar_lote(lote) for lote in lotes], ignore_index=True)


# Mostrar el DataFrame final
resultados

,ID,COMENTARIO,SENTIMIENTO,TOPICO,CRITERIO
0,1,"El producto llegó antes de lo esperado, muy sa...",Positivo,Instalación/Entrega,Entrega rápida
1,2,"El servicio no fue lo que esperaba, bastante d...",Negativo,Calidad de servicio al cliente,Servicio deficiente
2,3,"Las promociones son atractivas, pero el proces...",Neutral,Facturación,Proceso de compra complejo
3,4,"La atención al cliente fue excelente, muy amables",Positivo,Calidad de servicio al cliente,Atención amable y eficiente
4,5,"El producto es funcional, aunque no destaca en...",Neutral,Otros,Producto básico
5,6,No recibí una respuesta clara a mi problema,Negativo,Calidad de servicio al cliente,Respuesta no satisfactoria
6,7,"La calidad es buena, pero el precio es elevado",Neutral,Precio/Tarifas,Precio alto para la calidad
7,8,"Recibí exactamente lo que pedí, sin sorpresas",Positivo,Instalación/Entrega,Entrega precisa
8,9,Los precios son razonables para la calidad ofr...,Positivo,Precio/Tarifas,Buena relación calidad-precio
9,10,"El soporte fue útil, aunque tomó tiempo",Neutral,Calidad de servicio al cliente,Soporte útil pero lento
